# Возьмем готовую разбивку номенклатур по категориям

In [15]:
import pandas
import numpy as np
import codecs
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
# from sklearn import cross_validation, grid_search, linear_model, metrics

colnames = ['name', 'id', 'class']
data = pandas.read_csv('nameCatIdCatName.csv', names=colnames, encoding='utf8')
names = data['name'].tolist()
classes = data['class'].tolist()

# Найдем параметры регресии

In [16]:
vectorizer =  TfidfVectorizer()
corpus = vectorizer.fit_transform(names)
clf = LogisticRegressionCV(n_jobs=-1)
clf.fit(corpus, classes)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

# Построим по ним модель и проверим

In [17]:
train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(names, classes, 
                                                                                     test_size = 0.3, 
                                                                                     random_state = 0)

In [18]:
clf = LogisticRegression(C=10.0, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
           solver='lbfgs', tol=0.0001)
log = Pipeline([('vectorizer', TfidfVectorizer()),
                      ('classifier', clf)])
log.fit(train_data, train_labels)

Pipeline(steps=[('vectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_id...enalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

In [19]:
score = log.score(test_data, test_labels)
print(score)

0.930218805441


# Полученная точность 93%

In [ ]:
# from scipy.stats import randint as sp_randint
# log_params = {
# 'classifier__solver' : ['newton-cg', 'sag', 'lbfgs'],
# 'classifier__max_iter' : [50, 100, 200],
# 'classifier__C': [0.001, 0.1, 1, 10, 100]
# }
# cv = cross_validation.StratifiedShuffleSplit(train_labels, n_iter = 10, test_size = 0.2, random_state = 0)
# grid_cv = grid_search.GridSearchCV(log, log_params, scoring = 'accuracy', cv = cv)
# grid_cv.fit(train_data, train_labels)
# print grid_cv.best_score_
# print grid_cv.best_params_
# 0.854

# Раз уж мы тут все собрались, то посмотрим, что все это значит для наших  номенклатур

In [20]:
content = codecs.open("nom3utf.csv", 'r', 'utf8')
real_test = np.random.choice(np.array(content.readlines()), size = 11)

# Логистическая регрессия:

In [21]:
predicts = log.predict(real_test)
for x, p in zip(real_test, predicts):
    print(x + "  " + p)

Презервативы Durex N3 Elite (ультратонкие)
  ПЛАНИРОВАНИЕ СЕМЬИ
Омизак капс. киш./раст. 20мг бл. N30 Torrent Pharm.
  ЛЕКАРСТВА ПО РЕЦЕПТАМ
ЧИСТАЯ ЛИНИЯ фитомаска д/волос красот и сил 200мл Концерн Калина
  УХОД ЗА ВОЛОСАМИ
Кефир VIVO пор. 0.5г фл. N4 ГИ природных ресурсов
  ЛЕКАРСТВА ПО РЕЦЕПТАМ
Слабикап капли д/приема вн. 7.5мг/мл 10мл фл. Синтез
  ЛЕКАРСТВА ПО РЕЦЕПТАМ
Крем-мыло РБА Лесные ягоды 300мл
  ГИГИЕНА
МИР ДЕТСТВА клеенка 3-х слойн. [19534] Sun Bond Int.Company
  МАМА И МАЛЫШ
УМН.ЭМАЛЬ Коррекция неоднородных ногтей 15мл
  УХОД ЗА ТЕЛОМ
ХИМАЛАЯ ХЕРБАЛС Маска отшелушив. с миндалем и огурцом 150 мл.
  УХОД ЗА ЛИЦОМ
Королева красоты фиточай 2г ф-пак. N20 Здоровье нации
  НАРОДНЫЕ СРЕДСТВА
АкваМастер спрей наз. 0.65% 50мл с нас.-распыл. бут. Эвалар
  СРЕДСТВА ОТ ПРОСТУДЫ


# Ну, не так уж и плохо